In [96]:
import pandas as pd
from datetime import date, datetime
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import glob

In [97]:
file_path = "data/YTD files"
csv_files = glob.glob(os.path.join(file_path, '*.csv'))
print(csv_files)

['data/YTD files/2023 Profit and Loss YTD.csv', 'data/YTD files/2023 Profit and Loss.csv']


In [98]:
latest_file = max(csv_files, key=os.path.getmtime)

In [99]:
df = pd.read_csv(latest_file, index_col = False)

In [100]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,BJ WIN HOSPITALITY INC.,NaN,NaN,NaN,NaN
1,Profit and Loss,NaN,NaN,NaN,NaN
2,"From January 1, 2023 to September 30, 2023",NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,Year to Date,%,NaN,NaN


In [101]:
datestr = df.loc[2, 'Unnamed: 0']

# datestr = datestr.lower().replace(',', '')
split_date = datestr.split()

from_date = ' '.join(split_date[1:4])
input_from_date = datetime.strptime(from_date, "%B %d, %Y")
formatted_from_date = input_from_date.strftime("%m/%d/%y")

to_date = ' '.join(split_date[5:8])
input_to_date = datetime.strptime(to_date, "%B %d, %Y")
formatted_to_date = input_to_date.strftime("%m/%d/%y")

date_range = formatted_from_date + '-' + formatted_to_date
print(date_range)
# print(split_date[1:4].append(split_date[5:7]))

01/01/23-09/30/23


In [102]:
cols_to_remove = ['Unnamed: 3', 'Unnamed: 4']
df = df.drop(columns = cols_to_remove)

In [103]:
modcols = {
    'Unnamed: 0' : 'metric_type',
    'Unnamed: 1' : date_range,
    'Unnamed: 2' : 'percent',
}

df = df.rename(columns = modcols)

In [104]:
df = df.dropna(axis=0)
df = df.reset_index(drop=True)

In [105]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df)

In [106]:
# All modifications of dataframe
df.iloc[8, df.columns.get_loc('metric_type')] = 'PAYROLL TAXES OPS'
df.iloc[39, df.columns.get_loc('metric_type')] = 'PAYROLL TAXES ADMIN'
paranthesis = df[date_range].str.contains(r'\(|\)')
df.loc[paranthesis, date_range] = df.loc[paranthesis, date_range].str.replace(r'\(', '-').str.replace(r'\)', '', regex=True)
df[date_range] = df[date_range].str.replace('[\$,]', '', regex=True).astype(float)



/var/folders/nc/1czn_dfn327cnn5jm9s4xv7w0000gn/T/ipykernel_69606/590842208.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[paranthesis, date_range] = df.loc[paranthesis, date_range].str.replace(r'\(', '-').str.replace(r'\)', '', regex=True)


In [107]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df)

In [108]:
# columns for cost: total cost of sales, officer/admin salaries, payroll taxes, commercial ins, telephone, internet, property tax, 
# credit card commission, maint salary, payroll tax, pest, eqpt rep, fire and safety, swimming pool, maint&serv, plumbing, elec, trash&waste, 
# maint supp, total utilities, auto/gen insurance, building ins, 
rows_to_sum = ['Total Cost Of Sales', 'OFFICERS SALARIES', 'ADMINISTRATIVE SALARIES', 'PAYROLL TAXES ADMIN', 'COMMERCIAL INSURANCE', 
               'TELEPHONE', 'INTERNET & COMPUTER', 'PROPERTY TAXES', 'AUTO INSURANCE']
df['metric_type'] = df['metric_type'].str.lower().str.strip()
rows_to_sum = [row.lower().strip() for row in rows_to_sum]
# print(rows_to_sum)
temp = df['metric_type'].isin(rows_to_sum)
selectRows = df[temp]
# print(selectRows)
totalOpCostCurrYr = selectRows[date_range].sum()

In [109]:
print(totalOpCostCurrYr)

2087367.75


In [110]:
roomsSold = input("How many rooms sold?")
# retrieve total rooms sold from VM or NA reports
# e.g: 2023 total rooms YTD = 26076
cPORCurrYr = totalOpCostCurrYr/int(roomsSold)
print(f"{date_range}: ${round(cPORCurrYr, 2)}")

How many rooms sold? 26076


01/01/23-09/30/23: $80.05
